# Metrics of matching (simple)
Example of the functions to plot match_metrics of matching.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pylab as plt

## Generate random data and add to catalog

In [ ]:
# For reproducibility
np.random.seed(1)

In [ ]:
from support import gen_cluster
input1, input2 = gen_cluster()

In [ ]:
from clevar import ClCatalog
c1 = ClCatalog('Cat1', ra=input1['RA'], dec=input1['DEC'], z=input1['Z'], mass=input1['MASS'],
            mass_err=input1['MASS_ERR'], z_err=input1['Z_ERR'])
c2 = ClCatalog('Cat2', ra=input2['RA'], dec=input2['DEC'], z=input2['Z'], mass=input2['MASS'],
            mass_err=input2['MASS_ERR'], z_err=input2['Z_ERR'])
# Format for nice display
for c in ('ra', 'dec', 'z', 'z_err'):
    c1[c].info.format = '.2f'
    c2[c].info.format = '.2f'
for c in ('mass', 'mass_err'):
    c1[c].info.format = '.2e'
    c2[c].info.format = '.2e'

## Match catalogs

In [ ]:
from clevar.match import ProximityMatch
from clevar.cosmology import AstroPyCosmology

match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'which_radius': 'max', # Case of radius to be used, can be: cat1, cat2, min, max
    'preference': 'angular_proximity', # options are more_massive, angular_proximity or redshift_proximity
    'catalog1': {'delta_z':.2,
                'match_radius': '1 mpc'
                },
    'catalog2': {'delta_z':.2,
                'match_radius': '10 arcsec'
                }
}

cosmo = AstroPyCosmology()
mt = ProximityMatch()
mt.match_from_config(c1, c2, match_config, cosmo=cosmo)

## Recovery rate
Compute recovery rates, they are computed in mass and redshift bins. There are several ways they can be displayed:
- Single panel with multiple lines
- Multiple panels
- 2D color map

In [ ]:
from clevar.match_metrics import recovery

In [ ]:
zbins = np.linspace(0, 2, 6)
mbins = np.logspace(13, 14, 5)

### Simple plot
The recovery rates are shown as a function of redshift in mass bins.
They can be displayed as a continuous line or with steps:

In [ ]:
ax = recovery.plot(c1, 'cross', zbins, mbins)
plt.show()
ax = recovery.plot(c1, 'cross', zbins, mbins, shape='line')
plt.show()

They can also be transposed to be shown as a function of mass in redshift bins.

In [ ]:
ax = recovery.plot(c1, 'cross', zbins, mbins,
              shape='line', transpose=True)

### Panels plots
You can also have a panel for each bin:

In [ ]:
f, axes = recovery.plot_panel(c1, 'cross', zbins, mbins)
f, axes = recovery.plot_panel(c1, 'cross', zbins, mbins, transpose=True)

### 2D plots

In [ ]:
ax = recovery.plot2D(c1, 'cross', zbins, mbins)
plt.show()
ax = recovery.plot2D(c1, 'cross', zbins, mbins,
                add_num=True, num_kwargs={'fontsize':15})

## Distances of matching
Here we evaluate the distance between the cluster centers and their redshifts. These distances can be shown for all matched clusters, or in bins:

In [ ]:
from clevar.match_metrics import distances

In [ ]:
ax = distances.central_position(c1, c2, 'cross', radial_bins=20, radial_bin_units='degrees')

In [ ]:
ax = distances.central_position(c1, c2, 'cross', radial_bins=20, radial_bin_units='degrees',
                                quantity_bins='mass', bins=mbins, log_quantity=True)

In [ ]:
ax = distances.central_position(c1, c2, 'cross', radial_bins=20, radial_bin_units='degrees',
                                quantity_bins='z', bins=zbins[::2], log_quantity=False)

In [ ]:
ax = distances.redshift(c1, c2, 'cross', redshift_bins=20, normalize='cat1')

In [ ]:
ax = distances.redshift(c1, c2, 'cross', redshift_bins=20, normalize='cat1',
                        quantity_bins='mass', bins=mbins, log_quantity=True)

In [ ]:
ax = distances.redshift(c1, c2, 'cross', redshift_bins=20, normalize='cat1',
                        quantity_bins='z', bins=zbins[::2], log_quantity=False)

## Scaling Relations

In [ ]:
from clevar.match_metrics import scaling

### Redshift plots

#### Simple plot

In [ ]:
ax = scaling.redshift(c1, c2, 'cross')

#### Color points by $\log(M)$ value

In [ ]:
ax, cb = scaling.redshift_masscolor(c1, c2, 'cross', add_err=True)

#### Color points by density at plot

In [ ]:
ax, cb = scaling.redshift_density(c1, c2, 'cross', add_err=True)

In [ ]:
ax, cb = scaling.redshift_density(c1, c2, 'cross', ax_rotation=45)

#### Evaluate metrics of the distribution

In [ ]:
fig, axes = scaling.redshift_metrics(c1, c2, 'cross')

In [ ]:
fig, axes = scaling.redshift_density_metrics(c1, c2, 'cross', ax_rotation=45)

#### Split data into mass bins

In [ ]:
fig, axes = scaling.redshift_masspanel(c1, c2, 'cross', add_err=True)

#### Split data into mass bins and color by density

In [ ]:
fig, axes = scaling.redshift_density_masspanel(c1, c2, 'cross', add_err=True)

#### Evaluate the distribution
See how the distribution of mass happens in each bin for one of the catalogs

In [ ]:
%%time
fig, axes = scaling.redshift_dist_self(c2, redshift_bins_dist=21,
                              mass_bins=[10**13.0, 10**13.2, 10**13.5, 1e15],
                              redshift_bins=4, shape='line',
                              fig_kwargs={'figsize':(15, 6)})

Compare with the distribution on the other catalog

In [ ]:
%%time
fig, axes = scaling.redshift_dist(c1, c2, 'cross', redshift_bins_dist=21,
                              mass_bins=[10**13.0, 10**13.2, 10**13.5, 1e15],
                              redshift_bins=4, shape='line',
                              fig_kwargs={'figsize':(15, 6)})

### Mass plots

#### Simple plot

In [ ]:
ax = scaling.mass(c1, c2, 'cross', add_err=True, add_fit=True, fit_bins1=30)

#### Color points by redshift value

In [ ]:
ax, cb = scaling.mass_zcolor(c1, c2, 'cross', add_err=True)

#### Color points by density at plot

In [ ]:
ax, cb = scaling.mass_density(c1, c2, 'cross', add_err=True)

#### Split data into redshift bins

In [ ]:
fig, axes = scaling.mass_zpanel(c1, c2, 'cross', add_err=True)

#### Split data into redshift bins and color by density

In [ ]:
fig, axes = scaling.mass_density_zpanel(c1, c2, 'cross', add_err=True)

#### Evaluate metrics of the distribution

In [ ]:
fig, axes = scaling.mass_metrics(c1, c2, 'cross')

In [ ]:
fig, axes = scaling.mass_density_metrics(c1, c2, 'cross', ax_rotation=45)

#### Evaluate the distribution
See how the distribution of mass happens in each bin for one of the catalogs

In [ ]:
%%time
fig, axes = scaling.mass_dist_self(c2, mass_bins_dist=21,
                              mass_bins=[10**13.0, 10**13.2, 10**13.5, 1e14, 1e15],
                              redshift_bins=4, shape='line',
                              fig_kwargs={'figsize':(15, 6)})

Compare with the distribution on the other catalog

In [ ]:
%%time
fig, axes = scaling.mass_dist(c1, c2, 'cross', mass_bins_dist=21,
                              mass_bins=[10**13.0, 10**13.2, 10**13.5, 1e14, 1e15],
                              redshift_bins=4, shape='line',
                              fig_kwargs={'figsize':(15, 6)})